In [ ]:
#| default_exp datasets

# VideoFrameDataset

In [ ]:
#| export
import torch.utils.data as td
from avcv.all import *
from fastcore.all import *

class VideoFrameDataset(td.Dataset):
    def __init__(self, path_to_video, img_preproc=None, out_dir=None, img_size=(416, 416), fps=10, ext='png'):
        if out_dir is None:
            vname = get_name(path_to_video)
            out_dir = f'/tmp/ple/VideoFrameDataset/extracted_video_for_inference/{vname}_fps_{fps}'
#         self.out_dir = out_dir
        store_attr('img_preproc, out_dir, img_size, fps, ext')
    
        self.video = mmcv.VideoReader(path_to_video)
        self.paths = self.get_images(path_to_video)
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        img = mmcv.imread(self.paths[idx])
        if self.img_preproc is not None:
            x = self.img_preproc(img)
            return dict(x=x, img=img)
        return dict(img=img)
    
    def get_images(self, path_to_video):
        if not self.check_if_extracted(path_to_video):
            _im = mmcv.imrescale(self.video[0], self.img_size)
            im_h, im_w = _im.shape[:2]
            mmcv.mkdir_or_exist(self.out_dir)
            cmd = f"ffmpeg -n  -i {path_to_video} -s {im_w}x{im_h} -vf fps={self.fps} {self.out_dir}/%06d.{self.ext}"
            print(cmd)
            os.system(cmd)
#         else:
#             print('Skip extracting using ffmpeg since files are already extracted')
        paths = glob(f'{self.out_dir}/*.{self.ext}')
        return list(sorted(paths))
    
    def check_if_extracted(self, path_to_video):
        exist_num = len(glob(f'{self.out_dir}/*.{self.ext}'))
        expected_num = len(self.video)*self.fps/self.video.fps
        exist_rate = exist_num/expected_num
        if exist_rate<0.9:
            print(f'{exist_rate=}')
        return exist_rate>0.9



In [ ]:
#| hide
video = VideoFrameDataset('/tmp/toanbd1_eating_smoking_phoneusage_distraction_drowsiness.mp4')

exist_rate=0.4711927335410727
ffmpeg -n  -i /tmp/toanbd1_eating_smoking_phoneusage_distraction_drowsiness.mp4 -s 416x234 -vf fps=10 /tmp/ple/VideoFrameDataset/extracted_video_for_inference/toanbd1_eating_smoking_phoneusage_distraction_drowsiness_fps_10/%06d.png


ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/anhvth8/.conda/envs/py38 --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/tmp/toanbd1_eating_smoking_phoneusage_distraction_drowsiness.mp4':
  Metadata:
    major_brand     : isom
  

In [ ]:
!nbdev_export

In [ ]:
from ple.datasets import VideoFrameDataset